# Testing the process

This notebook allows the user to test the process. This is useful if you are doing something a little spicy like not using the OCOD dataset and this is giving you errors.

In [1]:
from address_parsing_helper_functions import *

import sys


In [13]:
pd.read_csv('/app/enhance_ocod/data/'+'OCOD.csv', encoding_errors= 'ignore').rename(columns = lambda x: x.lower().replace(" ", "_"))

,title_number,property_address,admindistrict,county,region,postalcode
0,TT33,"Units 7 And 8, The Glenmore Centre, Honeywood ...",DOVER,KENT,SOUTH EAST,CT16 3FH
1,291860,88 Wickham Road and a moiety of the private st...,LEWISHAM,GREATER LONDON,GREATER LONDON,SE4 1NJ
2,336453,"Flat A to Flat E (inc) 134 and, 134 Petherton ...",HACKNEY,GREATER LONDON,GREATER LONDON,N5 2RT
3,310936,"218 Blackstock Road, London N5 1EA A new filed...",ISLINGTON,GREATER LONDON,GREATER LONDON,N5 1EA
4,434576,"the ground floor flat being C1 Albany, Piccadi...",CITY OF WESTMINSTER,GREATER LONDON,GREATER LONDON,W1V 9RF
...,...,...,...,...,...,...
2306,WYK823456,playing fields at Reinwood County Junior Infan...,KIRKLEES,WEST YORKSHIRE,YORKS AND HUMBER,HD3 4YL
2307,SYK659657,Commercial unit on the north side of Meadow Ga...,BARNSLEY,SOUTH YORKSHIRE,YORKS AND HUMBER,S73 0UN
2308,TGL265969,"St Augustines Primary School, Dunfield Road an...",LEWISHAM,GREATER LONDON,GREATER LONDON,SE6 3RD
2309,SYK517198,Warmsworth and Edlington National Spiritualist...,DONCASTER,SOUTH YORKSHIRE,YORKS AND HUMBER,DN4 9LT


In [2]:

args = sys.argv  

root_path = '/app/enhance_ocod/data/'


ocod_data = load_and_prep_OCOD_data(root_path + 'OCOD.csv')

all_entities = spacy_pred_fn(spacy_model_path = root_path+'spacy_cpu_model', ocod_data = ocod_data)
#all_entities = load_cleaned_labels(root_path + 'full_dataset_no_overlaps.json')
full_expanded_data = parsing_and_expansion_process(all_entities, expand_addresses = True)

del all_entities #memory management

ocod_data = post_process_expanded_data(full_expanded_data, ocod_data)

del full_expanded_data #memory management


Loading the spaCy model


/usr/local/lib/python3.8/dist-packages/spacy/util.py:865: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.4.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Adding the datapoint id and title number meta data to the property address
predicting over the OCOD dataset using the pre-trained spaCy model
extracting entity label text
Names Entity Recognition labelling complete


/usr/local/lib/python3.8/dist-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


In [3]:
from locate_and_classify_helper_functions import *
print("Load ONSPD")
# zip file handler  
zip = zipfile.ZipFile(root_path + 'ONSPD.zip')
# looks in the data folder for a csv file that begins ONSPD
#This will obviously break if the ONS change the archive structure
target_zipped_file = [i for i in zip.namelist() if re.search(r'^Data\/ONSPD.+csv$',i) ][0]
postcode_district_lookup = load_postocde_district_lookup(root_path + "ONSPD.zip", target_zipped_file)

print("Pre-process expanded ocod data")
ocod_data = preprocess_expandaded_ocod_data(ocod_data, postcode_district_lookup)
print("Load and pre-process the Land Registry price paid dataset")
#loads from a folder of price paid files
price_paid_df = load_and_process_pricepaid_data(root_path+'price_paid_files/', postcode_district_lookup)
print("Add in missing Local authority codes to the ocoda dataset")
ocod_data = add_missing_lads_ocod(ocod_data, price_paid_df)
print("Load and pre-process the voa business ratings list dataset")
voa_businesses = load_voa_ratinglist(root_path +'VOA_ratings.csv', postcode_district_lookup)

del postcode_district_lookup #for memory purposes

print("Match street addresses and buildings")
ocod_data = street_and_building_matching(ocod_data, price_paid_df, voa_businesses)



Load ONSPD


/app/enhance_ocod/locate_and_classify_helper_functions.py:90: DtypeWarning: Columns (18,31,40,45) have mixed types. Specify dtype option on import or set low_memory=False.
  postcode_district_lookup = pd.read_csv(f)[['pcds','oslaua','oa11','lsoa11', 'msoa11', 'ctry']]


Pre-process expanded ocod data
Load and pre-process the Land Registry price paid dataset
Add in missing Local authority codes to the ocoda dataset
Load and pre-process the voa business ratings list dataset


/app/enhance_ocod/locate_and_classify_helper_functions.py:465: DtypeWarning: Columns (1,2,6,12,13,17,22) have mixed types. Specify dtype option on import or set low_memory=False.
  voa_businesses =  pd.read_csv(file_path,


Match street addresses and buildings
replace the missing lsoa using street matching
replace the missing lsoa using building matching
insert newly ID'd LSOA and OA
update missing LSOA and OA for nested properties where at least one nested property has an OA or LSOA


In [9]:
filled_lsoa_list = []
i = 1
unique_lad_codes = ocod_data[ocod_data['street_name'].notnull() & ocod_data['street_number'].notnull() & ocod_data['lsoa11cd'].isnull()]['lad11cd'].unique()

for target_lad in unique_lad_codes:
        if print_lads: print(target_lad)
            
        if i%print_every==0: print("lad ", i, " of "+ str(round(len(unique_lad_codes), 3)))
        i = i+1
        
        #subset to the relevat rows within a single lad
        missing_lsoa_df = ocod_data[ocod_data['street_name'].notnull() & ocod_data['street_number'].notnull() & ocod_data['lsoa11cd'].isnull() & (ocod_data['lad11cd']==target_lad)].copy()
        missing_lsoa_df.loc[:,'street_number2'] = missing_lsoa_df.loc[:,'street_number'].str.replace(r"^.*(?=\b[0-9]+$)", "", regex = True).str.replace(r"[^\d]", "", regex = True)

        target_street_names = missing_lsoa_df['street_name2'].unique()

        temp_lsoa = pd.concat([
            #the price paid data with names changed
            price_paid_df[price_paid_df['street_name2'].isin(target_street_names )  & 
                                        (price_paid_df['lad11cd']==target_lad) ], 
        #voa data added in                
                        voa_businesses[(voa_businesses['lad11cd']==target_lad)]]
                                                        )[['street_name2', 'street_number', 'lsoa11cd', 'lad11cd']].dropna(axis = 0, how = 'any', inplace = False)

        temp_lsoa.loc[:,'street_number2'] = temp_lsoa.loc[:,'street_number'].str.replace(r"^.*(?=\b[0-9]+$)", "", regex = True).str.replace(r"[^\d]", "", regex = True)
        
        temp_lsoa  = create_all_street_addresses(temp_lsoa[temp_lsoa['street_name2'].isin(target_street_names ) & 
                                                temp_lsoa['street_number2'].notnull()], 
                                        target_lad, 
                                        ['street_name2', 'street_number2', 'lsoa11cd'])
        

        for target_road in target_street_names:
            #print(target_road)
            missing_lsoa_road = missing_lsoa_df[missing_lsoa_df['street_name2']== target_road ].copy()
            temp_road = temp_lsoa[temp_lsoa['street_name2'] ==target_road ]

            if len(temp_road)>0:
                missing_lsoa_road['lsoa11cd'] = [street_number_to_lsoa(temp_road, int(missing_lsoa_road.iloc[missing_lsoa_row]['street_number2'])) 
                                                for missing_lsoa_row 
                                                in range(0, len(missing_lsoa_road))]
                filled_lsoa_list = filled_lsoa_list + [missing_lsoa_road]

In [10]:
  filled_lsoa_list 

[]

In [16]:
#This takes some time
print('Sub-street matching, this takes some time')
ocod_data = substreet_matching(ocod_data, price_paid_df, voa_businesses)


Sub-street matching, this takes some time


In [17]:

del price_paid_df #for memory purposes
print('Add in businesses per oa and lsoa')
ocod_data = counts_of_businesses_per_oa_lsoa(ocod_data, voa_businesses)

print('Identify businesses using address matching')
ocod_data = voa_address_match_all_data(ocod_data, voa_businesses)

del voa_businesses #probably not necessary but still delete to save memory

print('Classification type 1')
ocod_data = classification_type1(ocod_data)
print('Classification type 2')
ocod_data = classification_type2(ocod_data)

print('Contract ocod dataset')
ocod_data = contract_ocod_after_classification(ocod_data, class_type = 'class2', classes = ['domestic'] )

print('Process complete saving the enchanced ocod dataset to ' + root_path + 'enhanced_ocod_dataset.csv')

Add in businesses per oa and lsoa
Identify businesses using address matching
address matched  50 lads of 320
address matched  100 lads of 320
address matched  150 lads of 320
address matched  200 lads of 320
address matched  250 lads of 320
address matched  300 lads of 320
Classification type 1
Classification type 2
Contract ocod dataset
Process complete saving the enchanced ocod dataset to /app/enhance_ocod/data/enhanced_ocod_dataset.csv


In [19]:
#subset the dataframe to only the columns necessary for the dataset and save
ocod_data.loc[:, ['title_number', 'within_title_id', 'within_larger_title', 'unique_id', 'unit_id', 'unit_type',
       'building_name', 'street_number', 'street_name', 'postcode', 'city',
       'district',  'region', 'property_address', 'oa11cd', 'lsoa11cd',
       'msoa11cd',  'lad11cd', 'class', 'class2']].rename(columns={'within_title_id':'nested_id',
                                                                  'within_larger_title':'nested_title'}).to_csv(root_path+'test_ouput.csv')